# Analisi del Sentiment sulle Traduzioni ASLLRP

Questo notebook analizza il sentiment delle traduzioni ottenute dal dataset ASLLRP.
I passaggi sono:

1. Caricare il file `utterances_with_translations.csv`.
2. Pulire i dati rimuovendo le righe dove la traduzione non è disponibile.
3. Calcolare il sentiment per ogni traduzione utilizzando VADER.
4. Classificare il sentiment in "Positivo", "Negativo" o "Neutro" in base a una soglia.
5. Visualizzare la distribuzione dei sentiment.
6. Salvare i risultati in un nuovo file CSV.


In [8]:
# Step 1: Importare le librerie necessarie
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import os
from collections import Counter
import matplotlib

# Imposta il backend di Matplotlib per evitare problemi di visualizzazione in alcuni ambienti
matplotlib.use("Agg")

In [9]:
# Step 2: Caricare e pulire i dati
# Percorso del file CSV
csv_path = "../data/processed/utterances_with_translations.csv"

# Carica il file CSV
df = pd.read_csv(csv_path)

# Mostra le prime righe e le informazioni sul DataFrame
# print("Prime 5 righe del dataset:")
# print(df.head())
print(f"\nColonne disponibili: {list(df.columns)}")
print(f"Numero totale di righe: {len(df)}")

# Rimuovi le righe dove la traduzione non è stata trovata, è vuota o non è una stringa valida
df_cleaned = df.dropna(subset=["caption"])
df_cleaned = df_cleaned[
    ~df_cleaned["caption"].str.contains(
        "Traduzione non trovata|Translation tag vuota", na=False
    )
]

print(f"\nNumero di righe dopo la pulizia: {len(df_cleaned)}")
# print("Prime 5 righe del dataset pulito:")
# print(df_cleaned.head())


Colonne disponibili: ['video_name', 'Source collection', 'caption']
Numero totale di righe: 2127

Numero di righe dopo la pulizia: 2118


In [10]:
# Identifica e visualizza le righe scartate
# Le righe scartate sono quelle presenti in 'df' ma non in 'df_cleaned'
# Possiamo trovarle prendendo l'indice di 'df' e rimuovendo gli indici di 'df_cleaned'
discarded_indices = df.index.difference(df_cleaned.index)
discarded_rows = df.loc[discarded_indices]

print("Visualizzazione delle righe scartate:")
if not discarded_rows.empty:
    print(discarded_rows)
else:
    print("Nessuna riga è stata scartata.")

Visualizzazione delle righe scartate:
        video_name         Source collection                 caption
30    45035462.mp4     Cory_2013-6-25_sc0106  Traduzione non trovata
31    45107545.mp4     Cory_2013-6-25_sc0106  Traduzione non trovata
32    45239274.mp4     Cory_2013-6-25_sc0106  Traduzione non trovata
33    45404906.mp4     Cory_2013-6-25_sc0106  Traduzione non trovata
1306   8396360.mp4      Cory_2013-6-27_sc109                     NaN
1311   8395965.mp4      Cory_2013-6-27_sc109                     NaN
1627  98357949.mp4          3-Ben-Voice-Life  Traduzione non trovata
1988   3166107.mp4  Jonathan_2012-11-27_sc96                     NaN
2076   7454155.mp4    Rachel_2011-12-08_sc46                     NaN


In [11]:
# Step 3: Analisi del Sentiment
# Inizializza VADER
analyzer = SentimentIntensityAnalyzer()

# Soglia per classificare come positivo o negativo
threshold = 0.34


# Funzione per calcolare il sentiment e classificare l'emozione
def get_sentiment_emotion(text):
    scores = analyzer.polarity_scores(text)
    compound_score = scores["compound"]

    if compound_score >= threshold:
        emotion = "Positive"
    elif compound_score <= -threshold:
        emotion = "Negative"
    else:
        emotion = "Neutral"

    return compound_score, emotion


# Applica la funzione al DataFrame
df_cleaned[["compound_score", "emotion"]] = df_cleaned["caption"].apply(
    lambda x: pd.Series(get_sentiment_emotion(x))
)

print("\nPrime 5 righe con punteggi di sentiment ed emozione:")
print(df_cleaned.head())


Prime 5 righe con punteggi di sentiment ed emozione:
     video_name      Source collection  \
0   2751812.mp4  Cory_2013-6-25_sc0106   
1  98592049.mp4  Cory_2013-6-25_sc0106   
2  98857459.mp4  Cory_2013-6-25_sc0106   
3  98881709.mp4  Cory_2013-6-25_sc0106   
4  98895873.mp4  Cory_2013-6-25_sc0106   

                                             caption  compound_score  emotion  
0            There's something strange about myself.         -0.2023  Neutral  
1                        I never lived in a house...          0.0000  Neutral  
2                             I was born in Florida.          0.0000  Neutral  
3  I lived there in a house with my parents and t...          0.0000  Neutral  
4  ... while I was growing up until I was six  an...          0.1779  Neutral  


In [12]:
# Step 4: Visualizzare la distribuzione dei sentiment
# Conta le occorrenze di ogni emozione
emotion_counts = Counter(df_cleaned["emotion"])

print("\nDistribuzione dei sentiment:")
print(f"Positive: {emotion_counts.get('Positive', 0)} esempi")
print(f"Negative: {emotion_counts.get('Negative', 0)} esempi")
print(f"Neutral: {emotion_counts.get('Neutral', 0)} esempi")

# Crea il grafico a barre
plt.figure(figsize=(10, 6))
bars = plt.bar(
    emotion_counts.keys(), emotion_counts.values(), color=["green", "red", "gray"]
)
plt.title("Distribuzione delle Emozioni nel Dataset ASLLRP")
plt.xlabel("Emozione")
plt.ylabel("Numero di Video")

# Aggiungi il numero sopra ogni barra
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{int(height)}",
        ha="center",
        va="bottom",
        fontsize=10,
    )

# Aggiungi la soglia usata come testo
plt.text(
    0.95,
    0.95,
    f"Threshold: |{threshold}|",
    transform=plt.gca().transAxes,
    fontsize=12,
    verticalalignment="top",
    horizontalalignment="right",
    bbox=dict(facecolor="white", alpha=0.5),
)

# Salva l'immagine
output_folder = "../reports/figures"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(
    output_folder, f"asllrp_emotions_distribution_{threshold}.png"
)
plt.savefig(output_path)

print(f"\nGrafico della distribuzione salvato in: {output_path}")
plt.show()


Distribuzione dei sentiment:
Positive: 586 esempi
Negative: 244 esempi
Neutral: 1288 esempi



Grafico della distribuzione salvato in: ../reports/figures/asllrp_emotions_distribution_0.34.png


/var/folders/5w/qb_kxxjs5lscg9_8tpttrzs40000gn/T/ipykernel_30889/1347181616.py:52: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [13]:
# Calcola le percentuali di positivi e negativi (escludendo i neutri)
positive_count = emotion_counts.get("Positive", 0)
negative_count = emotion_counts.get("Negative", 0)
total_non_neutral = positive_count + negative_count

if total_non_neutral > 0:
    positive_percentage = (positive_count / total_non_neutral) * 100
    negative_percentage = (negative_count / total_non_neutral) * 100

    print("\nPercentuale di sentiment (esclusi i neutri):")
    print(f"Positivi: {positive_percentage:.2f}%")
    print(f"Negativi: {negative_percentage:.2f}%")
else:
    print("\nNessun esempio positivo o negativo trovato per calcolare la percentuale.")


Percentuale di sentiment (esclusi i neutri):
Positivi: 70.60%
Negativi: 29.40%


In [14]:
# Step 5: Salvare i risultati
# Filtra solo i dati positivi e negativi
filtered_df = df_cleaned[df_cleaned["emotion"].isin(["Positive", "Negative"])]

# Seleziona le colonne da salvare
output_df = filtered_df[["video_name", "caption", "emotion"]]

# Percorso del file CSV di output
output_csv_path = f"../data/processed/asllrp_video_sentiment_data_{threshold}.csv"

# Salva il DataFrame in un file CSV
output_df.to_csv(output_csv_path, index=False)

print(
    f"\nFile CSV con i dati di sentiment (Positivi e Negativi) salvato con successo in: {output_csv_path}"
)
print(f"Numero di righe salvate: {len(output_df)}")


File CSV con i dati di sentiment (Positivi e Negativi) salvato con successo in: ../data/processed/asllrp_video_sentiment_data_0.34.csv
Numero di righe salvate: 830
